In [ ]:
#Задание 1

import functools

user_role = 'user'

def role_required(role: str):

    def check_role(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            if user_role == role:
                return f(*args, **kwargs)
            else:
                return 'Permission denied'

        return wrapper

    return check_role            

@role_required('admin')
def secret_source():
    '''func get access to resource'''
    return 'Permission accepted'

user_role = 'admin'
print(secret_source())

user_role = 'user'
print(secret_source())

Permission accepted
Permission denied


In [41]:
#Задание 2
DB_CACHE = {}


def cache(db_name: str, expire = 5): 

    def cache_dec(f):
        def wrapper(*args, **kwargs):
            key = f'{db_name}_{f.__name__}'
            if key not in DB_CACHE:
                DB_CACHE[key] = dict()
            thing, *_ = args
            if thing not in DB_CACHE[key] or DB_CACHE[key][thing][1] == 0:
                
                data = f(*args, **kwargs) #make db request here
                DB_CACHE[key][thing] = [data, expire]

                return data + f' from {db_name}, now cached with expire={DB_CACHE[key][thing][1]}'
            else:
                DB_CACHE[key][thing][1] -= 1
                return f'{DB_CACHE[key][thing][0]} cached in {db_name}, expire={DB_CACHE[key][thing][1]}'
                
        return wrapper

    return cache_dec


@cache('sqlite')
def get_info(thing:str):
    return f'Info about: {thing}'

@cache('postgre', expire = 30)
def get_info2(thing:str):
    return f'second Info about: {thing}'

@cache('postgre', expire = 3)
def get_info3(thing:str):
    return f'third Info about: {thing}'

#Делаем запрос к sqlitr
print(get_info('thing_1'))
print(get_info('thing_1'))
print(get_info('thing_1'))
print(get_info('thing_1'))
print(get_info('thing_1'))
print(get_info('thing_1'))
print(get_info('thing_1'))

#Добавляем запрос к другой БД
print('______________________')
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))

#Добавляем еще один запрос ко второй бд
print('______________________')
print(get_info3('thing_3'))
print(get_info3('thing_3'))
print(get_info3('thing_3'))
print(get_info3('thing_3'))
print(get_info3('thing_3'))
print(get_info3('thing_3'))

#Проверяем что предыдущий запрос корректно работает после добавления нового
print('______________________')
print(get_info2('thing_2'))
print(get_info2('thing_2'))
print(get_info2('thing_2'))


Info about: thing_1 from sqlite, now cached with expire=5
Info about: thing_1 cached in sqlite, expire=4
Info about: thing_1 cached in sqlite, expire=3
Info about: thing_1 cached in sqlite, expire=2
Info about: thing_1 cached in sqlite, expire=1
Info about: thing_1 cached in sqlite, expire=0
Info about: thing_1 from sqlite, now cached with expire=5
______________________
second Info about: thing_2 from postgre, now cached with expire=30
second Info about: thing_2 cached in postgre, expire=29
second Info about: thing_2 cached in postgre, expire=28
second Info about: thing_2 cached in postgre, expire=27
second Info about: thing_2 cached in postgre, expire=26
second Info about: thing_2 cached in postgre, expire=25
second Info about: thing_2 cached in postgre, expire=24
______________________
third Info about: thing_3 from postgre, now cached with expire=3
third Info about: thing_3 cached in postgre, expire=2
third Info about: thing_3 cached in postgre, expire=1
third Info about: thing_3 c

In [20]:
#Задание 3
class safe_write:
    '''context manager for safety writing to file
        works in both binary and text modes
        rollback file if exception occures
    '''
    def __init__(self, file_path, mode='a'):
        self.file_path = file_path
        self.mode = mode
        self.file_handle = None #file handler
        self.pos = 0    #position in open file

    def __enter__(self):
        if self.mode == 'a':
            self.file_handle = open(self.file_path, self.mode, encoding='utf-8') 
        elif self.mode == 'ab':
            self.file_handle = open(self.file_path, self.mode)
        else:
            raise RuntimeError(f'cannot open file with mode {self.mode}') 
            
        self.pos = self.file_handle.tell()
        return self

    def write(self, data):
        try:
            self.file_handle.write(data)
        except Exception:
            self.file_handle.truncate(self.pos)

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type:
            print(f'Во время записи в файл было возбуждено исключение {exc_type}')
        self.file_handle.close()


with safe_write('undertale.txt') as file:
    file.write('Язнаю,что ничего не знаю, но другие не знают и этого.')

try:
    with safe_write('undertale.txt') as file:
        file.write('Если ты будешь любознательным, то будешь много знающим.', flush=True)
        raise ValueError
except Exception:
    pass

with open('undertale.txt', encoding='utf-8') as file:
    print(f'Содержимое файла: "{file.read()}"')

Во время записи в файл было возбуждено исключение <class 'TypeError'>
Содержимое файла: "Язнаю,что ничего не знаю, но другие не знают и этого."
